# Section: Federated Learning

# Lesson: Introducing Federated Learning

Federated Learning is a technique for training Deep Learning models on data to which you do not have access. Basically:

Federated Learning: Instead of bringing all the data to one machine and training a model, we bring the model to the data, train it locally, and merely upload "model updates" to a central server.

Use Cases:

    - app company (Texting prediction app)
    - predictive maintenance (automobiles / industrial engines)
    - wearable medical devices
    - ad blockers / autotomplete in browsers (Firefox/Brave)
    
Challenge Description: data is distributed amongst sources but we cannot aggregated it because of:

    - privacy concerns: legal, user discomfort, competitive dynamics
    - engineering: the bandwidth/storage requirements of aggregating the larger dataset

# Lesson: Introducing / Installing PySyft

In order to perform Federated Learning, we need to be able to use Deep Learning techniques on remote machines. This will require a new set of tools. Specifically, we will use an extensin of PyTorch called PySyft.

### Install PySyft

The easiest way to install the required libraries is with [Conda](https://docs.conda.io/projects/conda/en/latest/user-guide/overview.html). Create a new environment, then install the dependencies in that environment. In your terminal:

```bash
conda create -n pysyft python=3
conda activate pysyft # some older version of conda require "source activate pysyft" instead.
conda install jupyter notebook
pip install syft
pip install numpy
```

If you have any errors relating to zstd - run the following (if everything above installed fine then skip this step):

```
pip install --upgrade --force-reinstall zstd
```

and then retry installing syft (pip install syft).

If you are using Windows, I suggest installing [Anaconda and using the Anaconda Prompt](https://docs.anaconda.com/anaconda/user-guide/getting-started/) to work from the command line. 

With this environment activated and in the repo directory, launch Jupyter Notebook:

```bash
jupyter notebook
```

and re-open this notebook on the new Jupyter server.

If any part of this doesn't work for you (or any of the tests fail) - first check the [README](https://github.com/OpenMined/PySyft.git) for installation help and then open a Github Issue or ping the #beginner channel in our slack! [slack.openmined.org](http://slack.openmined.org/)

In [2]:
import torch as th

In [4]:
x = th.tensor([1,2,3,4,5])
x

tensor([1, 2, 3, 4, 5])

In [5]:
y = x + x

In [6]:
print(y)

tensor([ 2,  4,  6,  8, 10])


In [3]:
import syft as sy

In [5]:
hook = sy.TorchHook(th)

In [9]:
th.tensor([1,2,3,4,5])

tensor([1, 2, 3, 4, 5])

# Lesson: Basic Remote Execution in PySyft

## PySyft => Remote PyTorch

The essence of Federated Learning is the ability to train models in parallel on a wide number of machines. Thus, we need the ability to tell remote machines to execute the operations required for Deep Learning.

Thus, instead of using Torch tensors - we're now going to work with **pointers** to tensors. Let me show you what I mean. First, let's create a "pretend" machine owned by a "pretend" person - we'll call him Bob.

In [10]:
bob = sy.VirtualWorker(hook, id="bob")

In [11]:
bob._objects

{}

In [12]:
x = th.tensor([1,2,3,4,5])

In [13]:
x = x.send(bob)

In [14]:
bob._objects

{20774588599: tensor([1, 2, 3, 4, 5])}

In [15]:
x.location

<VirtualWorker id:bob #objects:1>

In [16]:
x.id_at_location

20774588599

In [17]:
x.id

53013744349

In [18]:
x.owner

<VirtualWorker id:me #objects:0>

In [19]:
hook.local_worker

<VirtualWorker id:me #objects:0>

In [20]:
x

(Wrapper)>[PointerTensor | me:53013744349 -> bob:20774588599]

In [21]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [22]:
bob._objects

{}

# Project: Playing with Remote Tensors

In this project, I want you to .send() and .get() a tensor to TWO workers by calling .send(bob,alice). This will first require the creation of another VirtualWorker called alice.

In [23]:
# try this project here!

In [24]:
import torch as th
import syft as sy

# hook = sy.TorchHook(th)

In [25]:
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")

In [26]:
bob._objects, alice._objects

({}, {})

In [27]:
x = th.tensor([1,2,3,4,5])

In [28]:
x_ = x.send(bob)
y_ = y.send(alice)
x_, y_

((Wrapper)>[PointerTensor | me:65256570810 -> bob:2965398965],
 (Wrapper)>[PointerTensor | me:15052370908 -> alice:1818428988])

In [29]:
bob._objects, alice._objects

({2965398965: tensor([1, 2, 3, 4, 5])},
 {1818428988: tensor([ 2,  4,  6,  8, 10])})

In [30]:
x = x_.get()
y = y_.get()
x, y

(tensor([1, 2, 3, 4, 5]), tensor([ 2,  4,  6,  8, 10]))

In [31]:
bob._objects, alice._objects

({}, {})

In [32]:
x = th.tensor([1,2,3,4,5])

In [33]:
x_ptr = x.send(bob, alice)
x_ptr

(Wrapper)>[MultiPointerTensor]
	-> (Wrapper)>[PointerTensor | me:30823801215 -> bob:52782988019]
	-> (Wrapper)>[PointerTensor | me:37387074585 -> alice:27719551963]

In [34]:
x, y = x_ptr.get()
x, y

(tensor([1, 2, 3, 4, 5]), tensor([1, 2, 3, 4, 5]))

In [35]:
x = th.tensor([1,2,3,4,5]).send(bob, alice)
x.get(sum_results=True)

tensor([ 2,  4,  6,  8, 10])

# Lesson: Introducing Remote Arithmetic

In [36]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(bob)

In [37]:
x

(Wrapper)>[PointerTensor | me:49530391851 -> bob:18768254368]

In [38]:
y

(Wrapper)>[PointerTensor | me:24312811258 -> bob:99129301651]

In [39]:
z = x + y

In [40]:
z

(Wrapper)>[PointerTensor | me:97542672289 -> bob:36450319168]

In [41]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [42]:
z = th.add(x,y)
z

(Wrapper)>[PointerTensor | me:51088171196 -> bob:4316070462]

In [43]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [44]:
x = th.tensor([1.,2,3,4,5], requires_grad=True).send(bob)
y = th.tensor([1.,1,1,1,1], requires_grad=True).send(bob)

In [45]:
z = (x + y).sum()

In [46]:
z.backward()

(Wrapper)>[PointerTensor | me:67744688792 -> bob:409508533]

In [47]:
x = x.get()

In [48]:
x

tensor([1., 2., 3., 4., 5.], requires_grad=True)

In [49]:
x.grad

tensor([1., 1., 1., 1., 1.])

# Project: Learn a Simple Linear Model

In this project, I'd like for you to create a simple linear model which will solve for the following dataset below. You should use only Variables and .backward() to do so (no optimizers or nn.Modules). Furthermore, you must do so with both the data and the model being located on Bob's machine.

In [50]:
# try this project here!

In [51]:
inputs = th.tensor([[1., 1], [1, 0], [0, 1], [0, 0]], requires_grad=True).send(bob)
target = th.tensor([[1.], [1], [0], [0]], requires_grad=True).send(bob)

In [52]:
weights = th.tensor([[0.], [0.]], requires_grad=True).send(bob)

In [53]:
for i in range(20):
    preds = inputs.mm(weights)
    loss = ((preds - target)**2).sum()
    loss.backward()
    weights.data.sub_(weights.grad * 0.1)
    weights.grad *= 0
    print(loss.get().data)

tensor(2.)
tensor(0.5600)
tensor(0.2432)
tensor(0.1372)
tensor(0.0849)
tensor(0.0538)
tensor(0.0344)
tensor(0.0220)
tensor(0.0141)
tensor(0.0090)
tensor(0.0058)
tensor(0.0037)
tensor(0.0024)
tensor(0.0015)
tensor(0.0010)
tensor(0.0006)
tensor(0.0004)
tensor(0.0003)
tensor(0.0002)
tensor(0.0001)


# Lesson: Garbage Collection and Common Errors


In [54]:
bob = bob.clear_objects()

In [55]:
bob._objects

{}

In [56]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [57]:
bob._objects

{}

In [58]:
del x

In [59]:
bob._objects

{}

In [60]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [61]:
bob._objects

{}

In [62]:
x = "asdf"

In [63]:
bob._objects

{}

In [64]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [65]:
x

(Wrapper)>[PointerTensor | me:47851861073 -> bob:20438793798]

In [66]:
bob._objects

{}

In [67]:
x = "asdf"

In [68]:
bob._objects

{}

In [69]:
del x

In [70]:
bob._objects

{}

In [71]:
bob = bob.clear_objects()
bob._objects

{}

In [72]:
for i in range(1000):
    x = th.tensor([1,2,3,4,5]).send(bob)

In [73]:
bob._objects

{}

In [74]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1])

In [75]:
z = x + y

TensorsNotCollocatedException: You tried to call a method involving two tensors where one tensor is actually locatedon another machine (is a PointerTensor). Call .get() on the PointerTensor or .send(bob) on the other tensor.

Tensor A: [PointerTensor | me:87235817738 -> bob:84960787937]
Tensor B: tensor([1, 1, 1, 1, 1])

In [76]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(alice)

In [77]:
z = x + y

TensorsNotCollocatedException: You tried to call __add__ involving two tensors which are not on the same machine! One tensor is on <VirtualWorker id:bob #objects:0> while the other is on <VirtualWorker id:alice #objects:1>. Use a combination of .move(), .get(), and/or .send() to co-locate them to the same machine.

# Lesson: Toy Federated Learning

Let's start by training a toy model the centralized way. This is about a simple as models get. We first need:

- a toy dataset
- a model
- some basic training logic for training a model to fit the data.

In [78]:
from torch import nn, optim

In [79]:
# A Toy Dataset
data = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True)
target = th.tensor([[1.],[1], [0], [0]], requires_grad=True)

In [80]:
# A Toy Model
model = nn.Linear(2,1)

In [81]:
opt = optim.SGD(params=model.parameters(), lr=0.1)

In [82]:
def train(iterations=20):
    for iter in range(iterations):
        opt.zero_grad()

        pred = model(data)

        loss = ((pred - target)**2).sum()

        loss.backward()

        opt.step()

        print(loss.data)
        
train()

tensor(4.8821)
tensor(1.2717)
tensor(0.6754)
tensor(0.4226)
tensor(0.2704)
tensor(0.1735)
tensor(0.1115)
tensor(0.0717)
tensor(0.0461)
tensor(0.0297)
tensor(0.0191)
tensor(0.0124)
tensor(0.0080)
tensor(0.0052)
tensor(0.0034)
tensor(0.0022)
tensor(0.0014)
tensor(0.0009)
tensor(0.0006)
tensor(0.0004)


In [6]:
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")

In [84]:
data_bob = data[0:2].send(bob)
target_bob = target[0:2].send(bob)

In [85]:
data_alice = data[2:4].send(alice)
target_alice = target[2:4].send(alice)

In [86]:
datasets = [(data_bob, target_bob), (data_alice, target_alice)]

In [87]:
def train(iterations=20):

    model = nn.Linear(2,1)
    opt = optim.SGD(params=model.parameters(), lr=0.1)
    
    for iter in range(iterations):

        for _data, _target in datasets:

            # send model to the data
            model = model.send(_data.location)

            # do normal training
            opt.zero_grad()
            pred = model(_data)
            loss = ((pred - _target)**2).sum()
            loss.backward()
            opt.step()

            # get smarter model back
            model = model.get()

            print(loss.get())

In [88]:
train()

tensor(5.8042, requires_grad=True)
tensor(0.9207, requires_grad=True)
tensor(0.4798, requires_grad=True)
tensor(0.7160, requires_grad=True)
tensor(0.2406, requires_grad=True)
tensor(0.4712, requires_grad=True)
tensor(0.1552, requires_grad=True)
tensor(0.3105, requires_grad=True)
tensor(0.1037, requires_grad=True)
tensor(0.2071, requires_grad=True)
tensor(0.0707, requires_grad=True)
tensor(0.1398, requires_grad=True)
tensor(0.0491, requires_grad=True)
tensor(0.0955, requires_grad=True)
tensor(0.0346, requires_grad=True)
tensor(0.0660, requires_grad=True)
tensor(0.0247, requires_grad=True)
tensor(0.0460, requires_grad=True)
tensor(0.0179, requires_grad=True)
tensor(0.0324, requires_grad=True)
tensor(0.0130, requires_grad=True)
tensor(0.0230, requires_grad=True)
tensor(0.0096, requires_grad=True)
tensor(0.0165, requires_grad=True)
tensor(0.0071, requires_grad=True)
tensor(0.0119, requires_grad=True)
tensor(0.0053, requires_grad=True)
tensor(0.0086, requires_grad=True)
tensor(0.0039, requi

# Lesson: Advanced Remote Execution Tools

In the last section we trained a toy model using Federated Learning. We did this by calling .send() and .get() on our model, sending it to the location of training data, updating it, and then bringing it back. However, at the end of the example we realized that we needed to go a bit further to protect people privacy. Namely, we want to average the gradients BEFORE calling .get(). That way, we won't ever see anyone's exact gradient (thus better protecting their privacy!!!)

But, in order to do this, we need a few more pieces:

- use a pointer to send a Tensor directly to another worker

And in addition, while we're here, we're going to learn about a few more advanced tensor operations as well which will help us both with this example and a few in the future!

In [7]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [8]:
x = th.tensor([1,2,3,4,5]).send(bob)
x

(Wrapper)>[PointerTensor | me:29932099933 -> bob:99698182828]

In [10]:
x = x.send(alice)

In [11]:
bob._objects

{99698182828: tensor([1, 2, 3, 4, 5])}

In [12]:
alice._objects

{29932099933: (Wrapper)>[PointerTensor | alice:29932099933 -> bob:99698182828]}

In [13]:
y = x + x

In [14]:
y

(Wrapper)>[PointerTensor | me:23024756959 -> alice:82288867147]

In [15]:
bob._objects

{99698182828: tensor([1, 2, 3, 4, 5]),
 98051520467: tensor([ 2,  4,  6,  8, 10])}

In [16]:
alice._objects

{29932099933: (Wrapper)>[PointerTensor | alice:29932099933 -> bob:99698182828],
 82288867147: (Wrapper)>[PointerTensor | alice:82288867147 -> bob:98051520467]}

In [17]:
jon = sy.VirtualWorker(hook, id="jon")

In [18]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [19]:
bob._objects

{22470669639: tensor([1, 2, 3, 4, 5])}

In [20]:
alice._objects

{42001639542: (Wrapper)>[PointerTensor | alice:42001639542 -> bob:22470669639]}

In [21]:
x = x.get()
x

(Wrapper)>[PointerTensor | me:42001639542 -> bob:22470669639]

In [22]:
bob._objects

{22470669639: tensor([1, 2, 3, 4, 5])}

In [23]:
alice._objects

{}

In [24]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [25]:
bob._objects

{}

In [26]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [27]:
bob._objects

{51356365384: tensor([1, 2, 3, 4, 5])}

In [28]:
alice._objects

{12275545969: (Wrapper)>[PointerTensor | alice:12275545969 -> bob:51356365384]}

In [29]:
del x

In [30]:
bob._objects

{}

In [31]:
alice._objects

{}

# Lesson: Pointer Chain Operations

In [32]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [33]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [34]:
bob._objects

{80404524867: tensor([1, 2, 3, 4, 5])}

In [35]:
alice._objects

{}

In [36]:
x.move(alice)

(Wrapper)>[PointerTensor | me:52559255575 -> alice:52559255575]

In [37]:
bob._objects

{}

In [38]:
alice._objects

{52559255575: tensor([1, 2, 3, 4, 5])}

In [39]:
x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [40]:
bob._objects

{20398183413: tensor([1, 2, 3, 4, 5])}

In [41]:
alice._objects

{52559255575: tensor([1, 2, 3, 4, 5]),
 42594930619: (Wrapper)>[PointerTensor | alice:42594930619 -> bob:20398183413]}

In [42]:
x.remote_get()

(Wrapper)>[PointerTensor | me:55969557581 -> alice:42594930619]

In [43]:
bob._objects

{}

In [44]:
alice._objects

{52559255575: tensor([1, 2, 3, 4, 5]), 42594930619: tensor([1, 2, 3, 4, 5])}

In [45]:
x.move(bob)

(Wrapper)>[PointerTensor | me:55969557581 -> bob:55969557581]

In [46]:
x

(Wrapper)>[PointerTensor | me:55969557581 -> bob:55969557581]

In [47]:
bob._objects

{55969557581: tensor([1, 2, 3, 4, 5])}

In [48]:
alice._objects

{52559255575: tensor([1, 2, 3, 4, 5])}